In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer

In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="4,5,6,7"

In [4]:
basepath = "/data/jyk/aac_dataset/clotho/encodec/"
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large')

In [5]:
data_files = {"train": "csv/train_short.csv", "validation": "csv/valid_short.csv"}

In [6]:
raw_dataset = load_dataset("csv", data_files=data_files)

Found cached dataset csv (/root/.cache/huggingface/datasets/csv/default-8533483370f473b7/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d)
100%|██████████| 2/2 [00:00<00:00, 923.96it/s]


In [7]:
raw_dataset['train']

Dataset({
    features: ['file_path', 'caption'],
    num_rows: 19175
})

In [10]:
from data.collator import EncodecCollator
import numpy as np
import os

In [11]:
def preprocessing(example):
    path = example['file_path']
    encodec = np.load(os.path.join(basepath, path))
    if encodec.shape[0]>1022:
        encodec = encodec[:1022, :]
    attention_mask = np.ones(encodec.shape[0]+2)
    target_text = tokenizer(text_target=example['caption'])

    return {'input_ids': encodec , 'attention_mask': attention_mask, 'labels': target_text['input_ids'], 'decoder_attention_mask': target_text['attention_mask']}


In [12]:
train_dataset = raw_dataset['train'].map(preprocessing, num_proc=16)

In [13]:
train_dataset.set_format("np", columns=['input_ids', 'attention_mask', 'labels', 'decoder_attention_mask'])

In [32]:
valid_dataset = raw_dataset['validation'].map(preprocessing, num_proc=16)
valid_dataset.set_format("np", columns=['input_ids', 'attention_mask', 'labels', 'decoder_attention_mask'])

Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-8533483370f473b7/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d/cache-a3db71731640afd3_*_of_00016.arrow


In [16]:
from modeling.audiobart import AudioBartForConditionalGeneration
model = AudioBartForConditionalGeneration.from_pretrained('bart/model')

Some weights of AudioBartForConditionalGeneration were not initialized from the model checkpoint at bart/model and are newly initialized: ['model.encodec_embeddings.6.weight', 'model.encodec_embeddings.4.weight', 'model.encodec_embeddings.1.weight', 'model.encodec_embeddings.7.weight', 'model.encodec_embeddings.5.weight', 'model.encodec_embeddings.3.weight', 'model.encodec_embeddings.2.weight', 'model.encodec_embeddings.0.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [25]:
from utils import count_parameters
print(count_parameters(model))

414688256


In [17]:
data_collator = EncodecCollator(tokenizer=tokenizer, model=model, return_tensors="pt")

In [19]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

In [36]:
training_args = Seq2SeqTrainingArguments('summary_test', per_gpu_train_batch_size=16)

In [37]:
trainer = Seq2SeqTrainer(
    model, training_args, train_dataset=valid_dataset, eval_dataset=valid_dataset, data_collator=data_collator, tokenizer=tokenizer
)

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
